## Downscaling ECOSTRESS LST using Sentinel-2 VSWIR products at High Resolution (<20m)

Notebook : Quentin Dehaene, mentored by Glynn Hulley    
Original Python Implementation : [Radoslaw Guzinski](https://github.com/radosuav/pyDMS)  
Original Implemenation : [Gao et al.](https://doi.org/10.3390/rs4113287)

Questions : quentin.dehaene@jpl.nasa.gov   

In [ ]:
# Import cell
from osgeo import gdal
import rasterio
import numpy as np
import os
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterio.mask
import rasterio.windows
import random
import datetime
import time
from datetime import datetime
from sentinelhub import (SHConfig, DataCollection, SentinelHubCatalog, SentinelHubRequest, BBox, bbox_to_dimensions, CRS, MimeType, Geometry,MosaickingOrder)
import rioxarray as rxr
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.animation as animation
from matplotlib import rc
from pyDMS_master.run_pyDMS import *

# If you recieve a No module named '...' error, it is likely because you haven't installed all the necessary packages (cf tutorial)

In this notebook, HR will refer to the high-resolution image used to train our regression algorithm. LR will refer to the coarse resolution image that we are trying to upsample, in our case the ECOSTRESS LST at 70m.

The first step is to reproject and subset LR to HR Coordinates Reference System (CRS) and extent. This means that the result, and all images in the process will be in the HR CRS and will share its extent. Even if it implies padding a smaller LR with nodata values to match the extents. Reversely, all the images bigger than HR would be cut.

The second step consists in resampling HR to the coarse resolution of LR (producing a temporary file). In the process, we compute the homogeneity inside each coarse resolution pixel. We are computing here how “pure” each coarse pixel is, i.e. how different are high resolution pixels that compose the coarse resolution pixel. For instance, if a coarse pixel is on a field, then it is very likely that all the high-resolution pixels composing that coarse pixel will be very similar because the material will be homogenous. However, in cities, larger pixels may actually be composed of several materials (asphalt roof, sidewalk, vegetation) and therefore be far less homogenous. We’ll use this homogeneity as a weight factor in our training.

The default threshold is 80%, meaning that the 20% of pixels that are the least homogenous will be disregarded for the training.

We can now start the training and fit the resampled HR to the LR.

For those interested in how the tree is conceived:

We are training a bagging of an ensemble of regression trees. Each of these trees is slightly more complicated than the usual regression tree. The principle of the tree is classic: each node is built to minimize the mean square error (MSE), hence each rule is made to split the data into different categories (the number of leaves being a parameter chosen here) and then we affect a value to each datum (each pixel in our case) falling into that category. The difference here lays in the way we determine the target value on each leaf: instead of giving the average of the y values (i.e. the LST) to all features in the leaf, we apply a Bayesian linear regression. We also have a limit as to how far from the training samples we can extrapolate when we predict with our regression tree.

The tree is thus trained using the resampled HR as $X_{train}$ and the reprojected LR as $y_{train}$ (ground truth). Our tree is trained at low resolution, because it is at this resolution that we have what we know to be true and that we can actually establish a link between reflectance and LST.

Now that the tree is trained, we can use it to make predictions. For that, we use the provided HR as X. We obtain a first prediction of the LST ($y_{pred}$) at the high resolution.

Then, comes the residual analysis: We compute the residual (y-$y_{pred}$) at the coarse resolution. We compare our predicted LST, resampled at the coarse resolution, to the original LR. For that, we don’t actually compare the LST but the $LST^4$ which is proportional to the exitance. It makes more sense, since physically, temperature doesn’t have to be conserved in the sharpening, the energy does. And sensors don’t measure temperature but radiance.

The final step is to smooth the residual, resample it to the high resolution, and sum it with the $y_{pred}$, our LST predicted. We now have a final LST prediction that verifies that the average radiance of all the high-resolution pixels composing a coarse pixel is equal to the radiance in the original LST.

This is our downscaled image that we were looking for.

I would like to note that there are two other sharpening techniques implemented in the pyDMS code, developed (and still being developed as of July 2024) by R. Guzinski. All the steps are the same, the only difference is the regression tree itself. There is on one hand, the Neural Network regressor i.e. instead of the tree presented here, we are using an MLP tree from [scikit-learn](https://github.com/aigamedev/scikit-neuralnetwork). On the other hand, is the one that’s actually closer to the original proposition of Gao, it is based on a [Cubist](https://www.rulequest.com/cubist-unix.html) regressor.

The results don’t differ significantly from what I have tried, I went to the oldest/default (and hence more fool-proofed) but I welcome any feedback from your end.


## Setting up and preprocessing

This whole notebook is actually a wrap over the original run_pyDMS. It is intended to render the sharpening easy and automatic by dealing with an entire folder at a time. Indeed, when interested in an area, we are looking at a series of images, a heatwave week, a summer month or more. The point here is to have all the files from your [AppEEARS](https://appeears.earthdatacloud.nasa.gov/) request in the same folders : one folder for the Land Surface Temperature files and another for the Quality Control files. They will all be downscaled using a single Sentinel 2 VSWIR image after being preprocessed earlier in the code.  
The output files, will all be written in one folder in the GEOTIFF format, that you can use in any GIS software. For each image a residual GEOTIFF will also be produced, but in most cases you can ignore these files.

For convenience, all of the inputs requested from the user are grouped the next 3 cells.

This cell is made for you to type the directories you want the dowloaded products and results to be written into, you also have to indicate the path towards the ECOSTRESS files dowloaded previously. When you type a directory, it shouldn't include the final / or \ (otherwise you will get a syntax error).

In [ ]:
# Choose your output folder for the downloaded Sentinel-2 products
s2_output_folder = r''
# Folder where all the ECOSTRESS Quality Control files are located. This can remain empty if you don't have any QC file
QC_dir = r''
# Folder where all the ECOSTRESS Cloud mask files are located. It is required to download and the the cloud mask if you are using collection 2. 
cloud_dir = r''
# Folder where all the ECOSTRESS LST files are located for the scene of interest
lst_dir = r''
# If your ECOSTRESS LST files are already scaled, then you can type here the directory in which they are located. If they are not scaled, you can leave this empty, they'll be scaled and placed in a directory in a following cell.
lst_dir_sc = r''
# Folder where all the sharpened ECOSTRESS LST files will be written for the scene of interest
dst_dir = r''

### Downloading Sentinel 2 product

All the Sentinel data is free to access, but it requires to create an account to download data.
First, if you don't have an account on [Copernicus Data Space](https://dataspace.copernicus.eu/), create one and log in.  
Now access your User Settings : My Account > DashBoard > User Settings (bottom left)  
Create a new OAuth client. And save your newly given token ID and password (called a secret here).  

If you encounter problem with the S2 download please refer to this [Documentation](https://sentinelhub-py.readthedocs.io/en/latest/index.html) or to this [Copernicus Web page](https://dataspace.copernicus.eu/news/2023-9-28-accessing-sentinel-mission-data-new-copernicus-data-space-ecosystem-apis).


Insert your logs for the OAuth Copernicus Data Space created above.

In [ ]:
config = SHConfig()
config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
config.sh_client_id ='' # type you client id
config.sh_client_secret ='' # type your client secret

Define your bounding box, its resolution and the time interval in which you want the data. The bounding box is limited to 2500 pixels, you will recieve an error if the request is larger.

In [ ]:
# The coordinates of the bounding box of your choosing, in lat lon : (xmin,ymin,xmax,ymax)
# Use the http://bboxfinder.com to find your box easily (already in the right order)
aoi_coords_wgs_84 = (-87.972336,41.769775,-87.606354,42.013591) # example

# Choose the resolution of the Sentinel data in meters 10 or 20, this will be the final resolution of the downscaled image
s2_res = 20

# Render the coordinates usable by the Copernicus API
aoi_bbox = BBox(bbox = aoi_coords_wgs_84,crs=CRS.WGS84)
aoi_size = bbox_to_dimensions(aoi_bbox,resolution = s2_res)

print(f"Image shape at {s2_res} m resolution: {aoi_size} pixels") # The size of the box is limited to 2500 pixels in each direction
if (aoi_size[0]>2499 or aoi_size[1]>2499) :
    raise(ValueError("The box is limited to 2500 pixels in each direction, try again with a smaller bounding box."))


# Choose your time interval (beginning, end) in the format (YYYY-MM-DD). You'll receive a S2 image from the tile with the lowest cloud coverage available in the interval :
interval = ("2023-06-01", "2023-08-01") # example

Download the S2 image with the previously defined parameters.

In [ ]:
# Request scripts, based off the sentinelhub documentation
# This script will be used to download all the S2 whose resolution is 20m or below
evalscript_all_bands_u20 = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B02","B03","B04","B05","B06","B07","B08","B8A","B11","B12"],
                units: "REFLECTANCE"

            }],
            output: {
                bands: 10,
                sampleType: "FLOAT32"
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B02,
                sample.B03,
                sample.B04,
                sample.B05,
                sample.B06,
                sample.B07,
                sample.B08,
                sample.B8A,
                sample.B11,
                sample.B12];
    }
"""
# This script will be used to download all the S2 whose resolution is 10m
evalscript_all_bands_u10 = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B02","B03","B04","B08"],
                units: 'REFLECTANCE'
            }],
            output: {
                bands: 4,
                sampleType: "FLOAT32"
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B02,
                sample.B03,
                sample.B04,
                sample.B08,];
    }
"""
# Request the data at 20m
if s2_res == 20 :
    request_all_bands_u20 = SentinelHubRequest(
        data_folder=s2_output_folder,
        evalscript=evalscript_all_bands_u20,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection = DataCollection.SENTINEL2_L2A.define_from("s2l2a", service_url=config.sh_base_url),
                time_interval = interval,
                mosaicking_order=MosaickingOrder.LEAST_CC, # selecting the tile in the interval with the least cloud coverage

            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=aoi_bbox,
        size=aoi_size,
        config=config,
    )
    resp = request_all_bands_u20.save_data(show_progress = True)
# Request the data at 10m
elif s2_res == 10 :
    request_all_bands_u10 = SentinelHubRequest(
        data_folder=s2_output_folder,
        evalscript=evalscript_all_bands_u10,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection = DataCollection.SENTINEL2_L2A.define_from("s2l2a", service_url=config.sh_base_url),
            time_interval = interval,
                mosaicking_order = MosaickingOrder.LEAST_CC
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=aoi_bbox,
        size=aoi_size,
        config=config,
    )

    resp = request_all_bands_u10.save_data(show_progress= True)
else :
    raise(ValueError('Unexpected resolution please change it to 10 or 20'))

# Rename the downloaded files for clarity
dirs = [d for d in os.listdir(s2_output_folder)]

most_recent_dir = max(dirs, key=lambda d: os.path.getmtime(os.path.join(s2_output_folder, d)))
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
new_name = f"S2_request_{timestamp}"
old_path = os.path.join(s2_output_folder, most_recent_dir)
new_path = os.path.join(s2_output_folder, new_name)
os.rename(old_path, new_path)
beg = interval[0]
end = interval [1]
for files in os.listdir(new_path):
    if files.__contains__('response') :
        os.rename(os.path.join(new_path,files),os.path.join(new_path,f"S2_{s2_res}m_{beg}_{end}.tif"))
        hr_img_path = os.path.join(new_path,f"S2_{s2_res}m_{beg}_{end}.tif") # This file will be used to train our model and downscale the ECOSTRESS image

### Preprocessing ECOSTRESS Data

In the following cells , we assume that you have the ECOSTRESS LST data products dowloaded alongside the Quality Control files if available (and cloud mask if you use Collection 2). Refer to the relative tutorial available [here](https://github.com/ECOSTRESS-Tutorials/) if you don't know how to obtain all the data you need. You can also use the alternate version of this notebook where it's all automatic.   

Preprocessing the QC files  

The QC files are coded in 16 bits and thus can't be easily seen as a mask file. For convience, we write new Quality Flag (QF) files that respresent only the last two bits of the QC files. Then, there are only four possible values:  
0 when the pixel is of best quality, 1 for nominal quality, 2 if a cloud is detected and 3 if the pixel is not produced. In the downscaling process, pixels with the last two values will be disregarded.   
For more information on the QC files : https://lpdaac.usgs.gov/documents/423/ECO2_User_Guide_V1.pdf (section 2.4)  

In [ ]:
for file in os.listdir(QC_dir) : 
    if not file.endswith('QF.tif') and not file.endswith('.xml') :
        file_qc = os.path.join(QC_dir,file)
        with rasterio.open(file_qc,'r') as f_qc :
            # Read the QC file, they are coded in 16 bits
            qc_img = f_qc.read((1)) 
            qc_img[qc_img==-99999] = -1  # Nodata values are read as -99999, we change it to -1 so that the last two bits appear as 11 (which means pixel not produced) and be masked out in the end
            # Select only the last two bits
            qc_img_2 = qc_img & 0b11 
            out_meta = f_qc.meta.copy()
        # Write the last two bits in a new file
        file_qf = file_qc.replace('.tif','_QF.tif')
        with rasterio.open(file_qf,'w',**out_meta) as dst :
            dst.write(qc_img_2,1)

Scaling the ECOSTRESS LST to normal Kelvin scale.  
  
The LST product is actually scaled at 0.02, the GIS software takes that scale in account before display so you might not see it if you directly display on QGIS or ArcGIS. However, in Python it's easier to apply the scale rather than reading the metadata. 
The newly scaled files will be stored in a subdirectory in the LST folder.  
  
Skip this cell if your ECOSTRESS LST data is already scaled (multi day aggregate for instance).

In [ ]:
# If the scaled subdirectory doesn't already exist, create it
lst_dir_sc = os.path.join(lst_dir,'scaled')
if not os.path.exists(lst_dir_sc) :
        os.mkdir(lst_dir_sc)

# Scale each file
for file in os.listdir(lst_dir) : 
        if file.endswith('.tif') :
                with rasterio.open(os.path.join(lst_dir,file),'r') as lr_img: 
                        out_image=lr_img.read().astype('float32')
                        out_image[out_image==0]=np.nan
                        out_meta = lr_img.meta

                out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        'dtype' :'float32'})

                dst_path = os.path.join(lst_dir_sc,file)
                with rasterio.open(dst_path,'w',**out_meta) as dst :
                        # Apply the scale 
                        dst.write(out_image*0.02 +0.49)

## Upsampling using pyDMS

The preprocessing is now over. Let's sharpen using pyDMS. Use one of the following cells depending on the presence of Quality Control files and the desired extent.

If you have QC files and used the cell above to process them into QF files, you can use one these two following cells :

If you use this cell, then the output extent will be the extent of the HR image. Thus, if the S2 image has a bigger extent than the ECOSTRESS image, then the edges will be padded with NaNs. If it is smaller then the sharpened image's extent will be the S2 image extent.

In [ ]:
useDecisionTree = True # You could change this to False if you want to use the Neural Network intead of the Decision tree, not recommended

files_sharpened = [] # list of the files sharpened

# Loop through the directory of LST images scaled
for file in os.listdir(lst_dir_sc) :
    if file.endswith('.tif') :
        if not os.path.exists(dst_dir) : # create the output directory if it doesn't exist already
                        os.mkdir(dst_dir)
        outputFilename = os.path.join(dst_dir,file.replace('.tif','_sharp_S2.tif')) # destination path for the sharpened images

        highResFilename = hr_img_path # the high resolution file is the downloaded s2 image
        lowResFilename = os.path.join(lst_dir_sc,file) # the low resolution file is the scaled LST image

        valid = False # Bool that states if a sence is "valid", not too cloudy or not presenting too many unproduced pixels (limit at 25% by default)
        thresh= 0.75 # you can modify this value between 0 (if you accept any file, the unusable pixels will be masked) and 1 (if you only want to sharpen files where every pixel is usable)
        
        # If the scene to be downscaled is a scene from Collection 2
        if file.__contains__('002') : 
            # The QC cloud bit being unreliable in Collection 2, we use the cloud mask directly as a validity mask
            file_cl = 'cloud_mask'.join(file.rsplit('LST', 1))
            lowResMaskFilename = os.path.join(cloud_dir,file_cl)
            lowresflags = [0]
            with rasterio.open(lowResMaskFilename,'r') as cld_msk : 
                cld_msk_arr = cld_msk.read(1)
                mask_sz = cld_msk_arr.size
                # Ensure that the scene to be sharpened isn't more than 25% cloudy
            if np.count_nonzero(cld_msk_arr)<(1-thresh)*mask_sz :
                valid = True
        # If the scene to be downscaled belongs to Collection 1        
        else : 
            # For Collection 1, we can rely on the QC (preprocessed earlier) file, no need to use the cloud mask
            file_qc = 'QC'.join(file.rsplit('LST', 1))
            file_qf = file_qc.replace('.tif','_QF.tif')
            lowResMaskFilename = os.path.join(QC_dir,file_qf)        
            with rasterio.open(lowResMaskFilename,'r') as mask :
                mask_array = mask.read(1)
                mask_sz = mask_array.size 
            lowresflags = [0,1]
            # Ensure that the scene to be sharpened isn't more than 25% cloudy or invalid
            if np.count_nonzero(mask_array==0) + np.count_nonzero(mask_array==1)>thresh*mask_sz :
                valid = True
        # Only downscale the files that are "valid" four our use case
        if valid :     
            commonOpts = {"highResFiles":               [highResFilename],
                            "lowResFiles":                [lowResFilename],
                            "lowResQualityFiles":         [lowResMaskFilename],
                            "lowResGoodQualityFlags":     lowresflags, # flags for acceptable pixels
                            "cvHomogeneityThreshold":     0, # the homogeneity threshold will be automatically computed
                            "movingWindowSize":           0,
                            "disaggregatingTemperature":  True} # we are dealing with LST
            dtOpts =     {"perLeafLinearRegression":    True,
                            "linearRegressionExtrapolationRatio": 0.25} # how much extrapolation we accept from the tree
            sknnOpts =   {'hidden_layer_sizes':         (10,),
                            'activation':                 'tanh'}
            nnOpts =     {"regressionType":             REG_sklearn_ann,
                            "regressorOpt":               sknnOpts}

            start_time = time.time()

            if useDecisionTree: #  Regression tree
                opts = commonOpts.copy()
                opts.update(dtOpts)
                disaggregator = DecisionTreeSharpener(**opts)
            else: # Neural network
                opts = commonOpts.copy()
                opts.update(nnOpts)
                disaggregator = NeuralNetworkSharpener(**opts)

            # Training the tree
            print("Training regressor...")
            disaggregator.trainSharpener()
            # Applying the newly trained tree to the LR image
            print("Sharpening...")
            downscaledFile = disaggregator.applySharpener(highResFilename, lowResFilename)
            # Performing residual analysis, to ensure that the unsharpened images's exitance is conserved
            print("Residual analysis...")
            residualImage, correctedImage = disaggregator.residualAnalysis(downscaledFile, lowResFilename,
                                                                            lowResMaskFilename,
                                                                            doCorrection=True)
            # Saving the image and its residual 
            print("Saving output...")
            highResFile = gdal.Open(highResFilename)
            if correctedImage is not None:
                outImage = correctedImage
            else:
                outImage = downscaledFile

            outFile = utils.saveImg(outImage.GetRasterBand(1).ReadAsArray(),
                                    outImage.GetGeoTransform(),
                                    outImage.GetProjection(),
                                    outputFilename)
            residualFile = utils.saveImg(residualImage.GetRasterBand(1).ReadAsArray(),
                                            residualImage.GetGeoTransform(),
                                            residualImage.GetProjection(),
                                            os.path.splitext(outputFilename)[0] + "_residual" +
                                            os.path.splitext(outputFilename)[1])
            files_sharpened.append(file)
            outFile = None
            residualFile = None
            downsaceldFile = None
            highResFile = None

            print(time.time() - start_time, "seconds")

If the ECOSTRESS image is completely included ( i.e. its extent is smaller) in the S2 image, then if desired, it is possible to cut the S2 image to the extent of the ECOSTRESS image, or to an extent of the user's choosing (which also has to be included). This might be useful for any mathematical postprocessing, such as computing RMSE or residual, because such an image would not contain any NaNs, and images would share same extents. To do so, use this cell :

In [ ]:
useDecisionTree = True # You could change this to False if you want to use the Neural Network intead of the Decision tree, not recommended

files_sharpened = [] #list of the files sharpened

# Loop through the directory of LST images
for file in os.listdir(lst_dir_sc) :
    if file.endswith('.tif') :
        if not os.path.exists(dst_dir) :  # create the output directory if it doesn't exist already
                        os.mkdir(dst_dir)
        outputFilename = os.path.join(dst_dir,file.replace('.tif','_sharp_S2_clipped.tif')) # destination path for the sharpened images
        lowResFilename = os.path.join(lst_dir_sc,file)
        lr_ds = rasterio.open(lowResFilename)

        projwin = [lr_ds.bounds.left,lr_ds.bounds.top,lr_ds.bounds.right,lr_ds.bounds.bottom] # cut to the extent of the LR image
        # projwin = [-118.08175, 34.16189, -117.998676, 34.112327] # example of a custom cutout over LA


        hr_img_path_clipped = hr_img_path.replace('.tif',f'_clipped.tif') # also produces a clipped version of the HR image
        ds = gdal.Open(hr_img_path)
        ds = gdal.Translate(hr_img_path_clipped, ds, projWin = projwin)
        ds = None
        lr_ds = None
        highResFilename = hr_img_path_clipped

        valid = False # Bool that states if a sence is "valid", not too cloudy or not presenting too many unproduced pixels (limit at 25% by default)
        thresh= 0.75 # you can modify this value between 0 (if you accept any file, the unusable pixels will be masked) and 1 (if you only want to sharpen files where every pixel is usable)
        
        # If the scene to be downscaled is a scene from Collection 2
        if file.__contains__('002') : 
            # The QC cloud bit being unreliable in Collection 2, we use the cloud mask directly as a validity mask
            file_cl = 'cloud_mask'.join(file.rsplit('LST', 1))
            lowResMaskFilename = os.path.join(cloud_dir,file_cl)
            lowresflags = [0]
            with rasterio.open(lowResMaskFilename,'r') as cld_msk : 
                cld_msk_arr = cld_msk.read(1)
                mask_sz = cld_msk_arr.size
                # Ensure that the scene to be sharpened isn't more than 25% cloudy
            if np.count_nonzero(cld_msk_arr)<(1-thresh)*mask_sz :
                valid = True
        # If the scene to be downscaled belongs to Collection 1        
        else : 
            # For Collection 1, we can rely on the QC (preprocessed earlier) file, no need to use the cloud mask
            file_qc = 'QC'.join(file.rsplit('LST', 1))
            file_qf = file_qc.replace('.tif','_QF.tif')
            lowResMaskFilename = os.path.join(QC_dir,file_qf)        
            with rasterio.open(lowResMaskFilename,'r') as mask :
                mask_array = mask.read(1)
                mask_sz = mask_array.size 
            lowresflags = [0,1]
            # Ensure that the scene to be sharpened isn't more than 25% cloudy or invalid
            if np.count_nonzero(mask_array==0) + np.count_nonzero(mask_array==1)>thresh*mask_sz :
                valid = True
        # Only downscale the files that are "valid" four our use case
        if valid :   
            commonOpts = {"highResFiles":               [highResFilename],
                            "lowResFiles":                [lowResFilename],
                            "lowResQualityFiles":         [lowResMaskFilename],
                            "lowResGoodQualityFlags":     lowresflags, # flags for acceptable pixels
                            "cvHomogeneityThreshold":     0, # the homogeneity threshold will be automatically computed
                            "movingWindowSize":           0,
                            "disaggregatingTemperature":  True} # we are dealing with LST
            dtOpts =     {"perLeafLinearRegression":    True,
                            "linearRegressionExtrapolationRatio": 0.25} # how much extrapolation we accept from the tree
            sknnOpts =   {'hidden_layer_sizes':         (10,),
                            'activation':                 'tanh'}
            nnOpts =     {"regressionType":             REG_sklearn_ann,
                            "regressorOpt":               sknnOpts}

            start_time = time.time()

            if useDecisionTree: #  Regression tree
                opts = commonOpts.copy()
                opts.update(dtOpts)
                disaggregator = DecisionTreeSharpener(**opts)
            else: # Neural network
                opts = commonOpts.copy()
                opts.update(nnOpts)
                disaggregator = NeuralNetworkSharpener(**opts)

            # Training the tree
            print("Training regressor...")
            disaggregator.trainSharpener()
            # Applying the newly trained tree to the LR image
            print("Sharpening...")
            downscaledFile = disaggregator.applySharpener(highResFilename, lowResFilename)
            # Performing residual analysis, to ensure that the unsharpened images's exitance is conserved
            print("Residual analysis...")
            residualImage, correctedImage = disaggregator.residualAnalysis(downscaledFile, lowResFilename,
                                                                            lowResMaskFilename,
                                                                            doCorrection=True)
            # Saving the image and its residual 
            print("Saving output...")
            highResFile = gdal.Open(highResFilename)
            if correctedImage is not None:
                outImage = correctedImage
            else:
                outImage = downscaledFile

            outFile = utils.saveImg(outImage.GetRasterBand(1).ReadAsArray(),
                                    outImage.GetGeoTransform(),
                                    outImage.GetProjection(),
                                    outputFilename)
            residualFile = utils.saveImg(residualImage.GetRasterBand(1).ReadAsArray(),
                                            residualImage.GetGeoTransform(),
                                            residualImage.GetProjection(),
                                            os.path.splitext(outputFilename)[0] + "_residual" +
                                            os.path.splitext(outputFilename)[1])
            files_sharpened.append(file)
            outFile = None
            residualFile = None
            downsaceldFile = None
            highResFile = None

            print(time.time() - start_time, "seconds")

You may wish to downscale images that don't have a QC or cloud mask file attached. For instance, if you're trying to sharpen an aggregated image such as a daytime average or if you're working with old images.


If you use this cell, then the output extent will be the extent of the HR image. Thus, if the S2 image has a bigger extent than the ECOSTRESS image, then the edges will be padded with NaNs. If it is smaller then the sharpened image's extent will be the S2 image extent.

In [ ]:
useDecisionTree = True # You could change this to False if you want to use the Neural Network intead of the Decision tree, not recommended

files_sharpened = [] #list of the files sharpened

# Loop through the directory of LST images
for file in os.listdir(lst_dir_sc) :
    if file.endswith('.tif') :
        if not os.path.exists(dst_dir) :  # create the output directory if it doesn't exist already
                        os.mkdir(dst_dir)
        outputFilename = os.path.join(dst_dir,file.replace('.tif','_sharp_S2.tif')) # destination path for the sharpened images
        lowResFilename = os.path.join(lst_dir_sc,file)
        highResFilename = hr_img_path
        lowResMaskFilename = ''


        commonOpts = {"highResFiles":               [highResFilename],
                        "lowResFiles":                [lowResFilename],
                        "lowResQualityFiles":         [lowResMaskFilename],
                        "lowResGoodQualityFlags":     [], #  no flags for acceptable pixels because no QF file to assert of the quality
                        "cvHomogeneityThreshold":     0, # the homogeneity threshold will be automatically computed
                        "movingWindowSize":           0,
                        "disaggregatingTemperature":  True} # we are dealing with LST
        dtOpts =     {"perLeafLinearRegression":    True,
                        "linearRegressionExtrapolationRatio": 0.25} # how much extrapolation we accept from the tree
        sknnOpts =   {'hidden_layer_sizes':         (10,),
                        'activation':                 'tanh'}
        nnOpts =     {"regressionType":             REG_sklearn_ann,
                        "regressorOpt":               sknnOpts}

        start_time = time.time()

        if useDecisionTree: # Regression tree
            opts = commonOpts.copy()
            opts.update(dtOpts)
            disaggregator = DecisionTreeSharpener(**opts)
        else: # Neural network
            opts = commonOpts.copy()
            opts.update(nnOpts)
            disaggregator = NeuralNetworkSharpener(**opts)

        # Training the tree
        print("Training regressor...")
        disaggregator.trainSharpener()
        # Applying the newly trained tree to the LR image
        print("Sharpening...")
        downscaledFile = disaggregator.applySharpener(highResFilename, lowResFilename)
        # Performing residual analysis, to ensure that the unsharpened images's exitance is conserved
        print("Residual analysis...")
        residualImage, correctedImage = disaggregator.residualAnalysis(downscaledFile, lowResFilename,
                                                                        lowResMaskFilename,
                                                                        doCorrection=True)
        # Saving the image and its residual 
        print("Saving output...")
        highResFile = gdal.Open(highResFilename)
        if correctedImage is not None:
            outImage = correctedImage
        else:
            outImage = downscaledFile

        outFile = utils.saveImg(outImage.GetRasterBand(1).ReadAsArray(),
                                outImage.GetGeoTransform(),
                                outImage.GetProjection(),
                                outputFilename)
        residualFile = utils.saveImg(residualImage.GetRasterBand(1).ReadAsArray(),
                                        residualImage.GetGeoTransform(),
                                        residualImage.GetProjection(),
                                        os.path.splitext(outputFilename)[0] + "_residual" +
                                        os.path.splitext(outputFilename)[1])
        files_sharpened.append(file)
        outFile = None
        residualFile = None
        downsaceldFile = None
        highResFile = None

        print(time.time() - start_time, "seconds")

If the ECOSTRESS image is completely included ( i.e. its extent is smaller) in the S2 image, then if desired, it is possible to cut the S2 image to the extent of the ECOSTRESS image, or to an extent of the user's choosing (which also has to be included) :

In [ ]:
useDecisionTree = True # You could change this to False if you want to use the Neural Network intead of the Decision tree, not recommended

files_sharpened = [] #list of the files sharpened

# Loop through the directory of LST images
for file in os.listdir(lst_dir_sc) :
    if file.endswith('.tif') :
        if not os.path.exists(dst_dir) :  # create the output directory if it doesn't exist already
                        os.mkdir(dst_dir)
        outputFilename = os.path.join(dst_dir,file.replace('.tif','_sharp_S2_clipped.tif')) # destination path for the sharpened images
        lowResFilename = os.path.join(lst_dir_sc,file)
        lr_ds = rasterio.open(lowResFilename)

        projwin = [lr_ds.bounds.left,lr_ds.bounds.top,lr_ds.bounds.right,lr_ds.bounds.bottom] # [xmin,ymax,xmax,ymin]
        # projwin = [-118.08175, 34.16189, -117.998676, 34.112327] # example of a custom cutout over LA  # this extent has to be fully included in the bounding box !
       


        hr_img_path_clipped = hr_img_path.replace('.tif',f'_clipped.tif') # also produces a clipped version of the HR image
        ds = gdal.Open(hr_img_path)
        ds = gdal.Translate(hr_img_path_clipped, ds, projWin = projwin)
        ds = None
        lr_ds = None

        highResFilename = hr_img_path_clipped
        lowResMaskFilename = ''


        commonOpts = {"highResFiles":               [highResFilename],
                        "lowResFiles":                [lowResFilename],
                        "lowResQualityFiles":         [lowResMaskFilename],
                        "lowResGoodQualityFlags":     [], #  no flags for acceptable pixels because no QF file to assert of the quality
                        "cvHomogeneityThreshold":     0, # the homogeneity threshold will be automatically computed
                        "movingWindowSize":           0,
                        "disaggregatingTemperature":  True} # we are dealing with LST
        dtOpts =     {"perLeafLinearRegression":    True,
                        "linearRegressionExtrapolationRatio": 0.25} # how much extrapolation we accept from the tree
        sknnOpts =   {'hidden_layer_sizes':         (10,),
                        'activation':                 'tanh'}
        nnOpts =     {"regressionType":             REG_sklearn_ann,
                        "regressorOpt":               sknnOpts}

        start_time = time.time()

        if useDecisionTree: # Regression tree
            opts = commonOpts.copy()
            opts.update(dtOpts)
            disaggregator = DecisionTreeSharpener(**opts)
        else: # Neural network
            opts = commonOpts.copy()
            opts.update(nnOpts)
            disaggregator = NeuralNetworkSharpener(**opts)

        # Training the tree
        print("Training regressor...")
        disaggregator.trainSharpener()
        # Applying the newly trained tree to the LR image
        print("Sharpening...")
        downscaledFile = disaggregator.applySharpener(highResFilename, lowResFilename)
        # Performing residual analysis, to ensure that the unsharpened images's exitance is conserved
        print("Residual analysis...")
        residualImage, correctedImage = disaggregator.residualAnalysis(downscaledFile, lowResFilename,
                                                                        lowResMaskFilename,
                                                                        doCorrection=True)
        # Saving the image and its residual 
        print("Saving output...")
        highResFile = gdal.Open(highResFilename)
        if correctedImage is not None:
            outImage = correctedImage
        else:
            outImage = downscaledFile

        outFile = utils.saveImg(outImage.GetRasterBand(1).ReadAsArray(),
                                outImage.GetGeoTransform(),
                                outImage.GetProjection(),
                                outputFilename)
        residualFile = utils.saveImg(residualImage.GetRasterBand(1).ReadAsArray(),
                                        residualImage.GetGeoTransform(),
                                        residualImage.GetProjection(),
                                        os.path.splitext(outputFilename)[0] + "_residual" +
                                        os.path.splitext(outputFilename)[1])
        files_sharpened.append(file)
        outFile = None
        residualFile = None
        downsaceldFile = None
        highResFile = None

        print(time.time() - start_time, "seconds")

### Display

Plot one random sharpened image

In [ ]:
# Pick one random sharpened file
file = random.choice(files_sharpened)
sharpened_file = os.path.join(dst_dir,file.replace('.tif','_sharp_S2.tif'))
raw_file = os.path.join(lst_dir_sc,file)
with rasterio.open(raw_file,'r') as lr_img :
    raw_lst = lr_img.read(1)
with rasterio.open(sharpened_file,'r') as shrp_img :
    shrp_lst = shrp_img.read(1)

custom_cmap = plt.colormaps['rainbow'].copy()
custom_cmap.set_under('black')

# Plot the ECOSTRESS original product
plt.figure(1)
plt.imshow(raw_lst,cmap=custom_cmap)
plt.axis('off')
plt.colorbar(label ='LST(K)')
vmin, vmax = plt.gci().get_clim() # save the limits to share them in the second figure
plt.title("ECOSTRESS LST 70m")
plt.show()

# Plot the ECOSTRESS sharpened product
plt.figure(2)
plt.imshow(shrp_lst,cmap=custom_cmap,clim =(vmin,vmax))
plt.axis('off')
plt.colorbar(label ='LST(K)')
plt.title("ECOSTRESS LST sharpened to 30m")
plt.show()

Create an animation of all the sharpened scenes

In [ ]:
def title_from_file(filename):
  time_info = filename.split('_')[4][3:]
  dt = datetime.strptime(time_info, '%Y%j%H%M%S')

  formatted_datetime = dt.strftime('%Y-%m-%d at %I:%M %p (UTC)')
  title = f"Land Surface Temperature (K)  {formatted_datetime} "

  return title

# Animation creation
rc('animation', html='jshtml')
fig = plt.figure()
ax = fig.add_subplot(111)

div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')

LST_sharpened_rasters = [rxr.open_rasterio(os.path.join(dst_dir,file.replace('.tif','_sharp_S2.tif'))).squeeze("band", drop=True) for file in files_sharpened]
print([os.path.join(dst_dir,file.replace('.tif','_sharp_S2.tif')) for file in files_sharpened])
custom_cmap = plt.colormaps['rainbow'].copy()
custom_cmap.set_under('black')


frames = []
for i in range(len(LST_sharpened_rasters)):
  LST_data = LST_sharpened_rasters[i].values
  lst_min = LST_data.min()
  np.nan_to_num(LST_data,copy = False)
  frames.append(LST_data)

cv0 = frames[0]
im = ax.imshow(cv0,cmap=custom_cmap) 
cb = fig.colorbar(im, cax=cax)
tx = ax.set_title(title_from_file(files_sharpened[i]))

def animate(i):
  arr = frames[i]
  vmax     = np.max(arr[arr>0.0])
  vmin     = np.min(arr[arr>0.0])
  im.set_data(arr)
  im.set_clim(vmin, vmax)
  tx.set_text(title_from_file(files_sharpened[i]))
  return im,tx


ani = animation.FuncAnimation(fig, animate, frames=len(files_sharpened),interval=1000,blit = True, repeat_delay=5000)
ani